In [ ]:
!pip install dagshub

In [6]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

from imblearn.combine import SMOTETomek

In [7]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
df['Glucose'] = df['Glucose'].replace(0, df['Glucose'].mean())
df['BloodPressure'] = df['BloodPressure'].replace(0, df['BloodPressure'].mean())
df['SkinThickness'] = df['SkinThickness'].replace(0, df['SkinThickness'].mean())
df['Insulin'] = df['Insulin'].replace(0, df['Insulin'].mean())
df['BMI'] = df['BMI'].replace(0, df['BMI'].mean())
df['DiabetesPedigreeFunction'] = df['DiabetesPedigreeFunction'].replace(0, df['DiabetesPedigreeFunction'].mean())

df['BMI'] = df['BMI'].astype('int64')
df['DiabetesPedigreeFunction'] = df['DiabetesPedigreeFunction'].astype('int64')

In [9]:

X = df.drop(columns=['Outcome'])
y = df['Outcome']

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=.20, random_state=42)

In [10]:
smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1], dtype=int64), array([386, 386], dtype=int64))

In [11]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(),
        {'C':1, 'solver':'liblinear'},
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(), 
        {'n_estimators':30, 'max_depth':3},
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(), 
        {'use_label_encoder':False, 'eval_metric':'logloss'},
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(), 
        {'use_label_encoder':False, 'eval_metric':'logloss'},
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [13]:
reports = []

for model_name, model, params, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [15]:
# Initialize MLflow
import mlflow
mlflow.set_experiment("Diabetes-MLOPS-Exp")

mlflow.set_tracking_uri('https://dagshub.com/UpscalewithLakhan/MLFlow-Exp-Tracking.mlflow')

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    params = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model") 

MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 403 != 200. Response body: ''

In [46]:
reports

[{'0': {'precision': 0.8037383177570093,
   'recall': 0.8686868686868687,
   'f1-score': 0.8349514563106796,
   'support': 99.0},
  '1': {'precision': 0.723404255319149,
   'recall': 0.6181818181818182,
   'f1-score': 0.6666666666666666,
   'support': 55.0},
  'accuracy': 0.7792207792207793,
  'macro avg': {'precision': 0.7635712865380792,
   'recall': 0.7434343434343434,
   'f1-score': 0.7508090614886731,
   'support': 154.0},
  'weighted avg': {'precision': 0.7750475811720592,
   'recall': 0.7792207792207793,
   'f1-score': 0.7748497457235322,
   'support': 154.0}},
 {'0': {'precision': 0.7719298245614035,
   'recall': 0.8888888888888888,
   'f1-score': 0.8262910798122066,
   'support': 99.0},
  '1': {'precision': 0.725,
   'recall': 0.5272727272727272,
   'f1-score': 0.6105263157894737,
   'support': 55.0},
  'accuracy': 0.7597402597402597,
  'macro avg': {'precision': 0.7484649122807017,
   'recall': 0.7080808080808081,
   'f1-score': 0.7184086978008402,
   'support': 154.0},
  'we